In [0]:
%run ./config

In [0]:
from pyspark.sql.functions import expr, rand, substring, lit, current_timestamp, col, date_format, add_months, current_date, trunc,  last_day
from pyspark.sql.types import IntegerType
from datetime import datetime

In [0]:
loaded_dt = datetime.now().strftime("%Y%m%d%H%M")
vehicle_instance_path = f'{path_prefix}/vehicle_instance'
sensor_data_path = f'{path_prefix}/sensor_data_path'

In [0]:
df = spark.range(1, 400001).withColumnRenamed("id", "vehicle_id")
columns_added_df = df.withColumn("model_id",(rand() * 1000).cast("int") + 1)\
                    .withColumn("variant_id",(rand() * 10).cast("int") + 1)\
                    .withColumn("owner_id",(rand() * 350000).cast("int") + 1)\
                    .withColumn("loaded_dt", lit(loaded_dt))

coalesce_df = columns_added_df.coalesce(1)

coalesce_df.write.mode("append").partitionBy("loaded_dt").option("header", "true").csv(vehicle_instance_path)  

In [0]:
month_end_date = (
    spark.range(0, 120) 
    .withColumn(
        "month_start",
        add_months(trunc(current_date(), "MONTH"), -120 + col("id"))
    )
    .withColumn(
        "recorded_date",
        last_day(col("month_start"))
    )
    .select("recorded_date")
    .orderBy("recorded_date")
)

vehcile_id_df = spark.range(1, 400001).withColumnRenamed("id", "vehicle_id")

final_df = month_end_date.crossJoin(vehcile_id_df)\
                    .withColumn("condition_desc",expr("substring('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', 1, 1 + cast(rand() * 20 as int))"))\
                    .withColumn("sensor_a",(rand() * 10).cast("int") + 1)\
                    .withColumn("sensor_b",(rand() * 10).cast("int") + 1)\
                    .withColumn("sensor_c",(rand() * 10).cast("int") + 1)\
                    .withColumn("sensor_d",(rand() * 10).cast("int") + 1)\
                    .withColumn("sensor_e",(rand() * 10).cast("int") + 1)\
                    .withColumn("sensor_f",(rand() * 10).cast("int") + 1)\
                    .withColumn("sensor_g",(rand() * 10).cast("int") + 1)\
                    .withColumn("sensor_h",(rand() * 10).cast("int") + 1)\
                    .withColumn("loaded_dt", lit(loaded_dt))\
                    .coalesce(1)


final_df.write.mode("append").partitionBy("loaded_dt").option("header", "true").csv(sensor_data_path)